In [1]:
from poker_utils.hands import get_possible_hands, find_blocked_hands, find_dominated_hands
from poker_utils.constants import HANDS_DICT, HANDS
from poker_utils.simulation import simulate_hand_randrange
import pandas as pd
import numpy as np
import torch
from treys import Evaluator
import networkx as nx
from itertools import product
from torch_geometric.data import Data, Dataset

In [3]:
evaluator = Evaluator()

In [4]:
hand_data = pd.read_csv("data/hand_data.csv")

In [5]:
hand_data = hand_data.astype({'avg_score.flop': int, 'avg_score.turn': int,'avg_score.river': int})

In [6]:
hand_data['score_flop'] = hand_data['avg_score.flop'].apply(
    lambda x: evaluator.class_to_string(evaluator.get_rank_class(x)))

hand_data['score_turn'] = hand_data['avg_score.turn'].apply(
    lambda x: evaluator.class_to_string(evaluator.get_rank_class(x)))

hand_data['score_river'] = hand_data['avg_score.river'].apply(
    lambda x: evaluator.class_to_string(evaluator.get_rank_class(x)))

In [7]:
hand_data['delta_win_perc'] = hand_data['win_perc.river'] - hand_data['win_perc.flop']

In [8]:
equity_data = pd.read_csv("data/equity_data.csv")

In [9]:
equity_data['hand1.hand'] = equity_data['hand1.hand'].apply(
        lambda x: x if x.endswith(('o', 's')) else x + 'o'
    )

equity_data['hand2.hand'] = equity_data['hand2.hand'].apply(
        lambda x: x if x.endswith(('o', 's')) else x + 'o'
    )

In [11]:
node_features = torch.rand((169, 3))  # Example with 3 features per node

In [12]:
# Define edges (example: connections based on domination)
edge_index = torch.tensor([
    [0, 1, 2],  # From nodes (hand indices)
    [1, 2, 3]   # To nodes (hand indices)
], dtype=torch.long)

In [13]:
edge_index = torch.tensor(list(product(range(169), range(169))), dtype=torch.long).T.contiguous()

In [14]:
dom_block_data = pd.read_csv("data/dom_block_data.csv")
ev_data = pd.read_excel("data/hand_ev9.xlsx")

In [15]:
dom_block_data = dom_block_data.set_index("hand")
ev_data = ev_data.set_index("hand")
hand_data = hand_data.set_index("hand")

In [16]:
hand_chart = pd.read_excel('data/hand_chart.xlsx')

In [17]:
G = nx.DiGraph()
for row in range(13):
    for col in range(13):
        hand = hand_chart.iloc[row, col]
        G.add_node(hand)
        

In [18]:
for source in G.nodes():
    blocked_hands_dict = find_blocked_hands(source)
    dominated_hands_dict = find_dominated_hands(source)
    for dest in G.nodes():
        blocks = 1.0 if dest in blocked_hands_dict else 0.0
        if blocks == 1.0:
            combinations_blocked = blocked_hands_dict[dest]
        else:
            combinations_blocked = 0.0
            
        dominates = 1.0 if dest in dominated_hands_dict else 0.0
        if dominates == 1.0:
            combinations_dominated = dominated_hands_dict[dest]
        else:
            combinations_dominated = 0.0

        G.add_edge(source, dest, 
                   blocks=blocks, 
                   dominates=dominates, 
                   combinations_blocked=combinations_blocked,
                   combinations_dominated=combinations_dominated)

In [19]:
# Edge features
# equity_data[['hand1.hand','hand2.hand','hand1.equity','hand2.equity']]
# Block DOm: combinations_blocked, combinations_dominated

In [20]:
edge_attributes = []
for i,j in edge_index.T:
    source_index = i.item()
    dest_index = j.item()
    source_hand = HANDS_DICT[source_index]
    dest_hand = HANDS_DICT[dest_index]
    edge_block_dom_dict = G.get_edge_data(source_hand, dest_hand)
    block_dom_data = [
        edge_block_dom_dict['combinations_blocked'], edge_block_dom_dict['combinations_dominated']
        ]
    edge_equity_data = equity_data[
        (equity_data['hand1.hand'] == source_hand) & 
        (equity_data['hand2.hand'] == dest_hand)
        ]
    if edge_equity_data.shape[0] == 0:
        edge_equity_data = equity_data[
            (equity_data['hand1.hand'] == dest_hand) & 
            (equity_data['hand2.hand'] == source_hand)
            ]
        equity_dat = edge_equity_data['hand2.equity'].tolist()
    else:
        equity_dat = edge_equity_data['hand1.equity'].tolist()
    
    #### REMOVE AFTER GETTING THIS DATA
    if len(equity_dat) == 0:
        equity_dat = [0]

    edge_data = block_dom_data + equity_dat
    edge_attributes.append(edge_data)
    

In [21]:
edge_attr = torch.tensor(np.array(edge_attributes), dtype=torch.float32)

In [22]:
# Node attribute Data:

# hand_data[['connectedness','suited','pair','win_perc.river','delta_win_perc']]
# dom_block_data[['dom_ratio','block_ratio_cat','dom_block_ratio','cluster']]
# ev_data (all) SB, BB, pos3, pos4, pos5, pos6, pos7, pos8, D

In [23]:
node_attributes = []
for hand_idx, hand in HANDS_DICT.items():
    basic_dat = hand_data[
        ['connectedness','suited','pair','win_perc.river','delta_win_perc']].loc[hand].tolist()
    
    dom_block_dat = dom_block_data[
        ['dom_ratio','block_ratio_cat','dom_block_ratio','cluster']].loc[hand].tolist()
    
    ev_dat = ev_data.loc[hand].tolist()
    node_attributes.append(basic_dat+dom_block_dat+ev_dat)
    

In [24]:
node_attr = torch.tensor(np.array(node_attributes), dtype=torch.float32)

In [25]:
# 

In [26]:
graph_data = Data(x=node_attr, edge_index=edge_index, edge_attr=edge_attr)

In [29]:
class PokerHandsGraphDataset(Dataset):
    def __init__(self, pyg_data, transform=None):
        super().__init__(transform=transform)
        self.data = pyg_data
        
    def len(self):
        return 1
        
    def get(self, idx):
        return self.data

In [30]:
d = PokerHandsGraphDataset(graph_data)

<bound method BaseData.node_attrs of Data(x=[169, 18], edge_index=[2, 28561], edge_attr=[28561, 3])>